In [114]:
import requests # this library handles http requests... for eg API GET requests.
import os # enables me to retrieve api key from another file
from dotenv import load_dotenv # allows me to access env file with api key
import pandas as pd
from geopy.geocoders import Nominatim
import json

load_dotenv("store.env") # loads data from env file



True

In [ ]:
versionnumber = "1.0"
keywords = "planner"
locationNames = ["unitedkingdom","anywhere"] # list of location Names
locationName = ",".join(locationNames) # joined list of names to create a comma seperate list. reed API also works with two location names listed "&locationName=''&locationName=''"
employerId = ""
distanceinmiles = ""
reed_api_key = os.getenv("reed_api_key") #retrieves api_key from .env file. This means i don't need to hard code my API key into script for improved security.

url = f"https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&locationName={locationName}&employerId={employerId}&distanceFromLocation={distanceinmiles}"




https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom,anywhere&employerId=&distanceFromLocation=


In [ ]:
response = requests.get(url, auth=(reed_api_key, ''))

if response.status_code == 200:
    # print(f"Response Success {response.status_code}")
    reed_data = response.json()
else:
    print(f"unsuccessful = {response.status_code}")



Response Success 200


In [ ]:
# Parse the JSON file using "results" as the key:

df = pd.DataFrame(reed_data["results"]) #"results" is the JSON key, so i have used this key to extract each job into the df.

# Data clean-up:
df.fillna("NaN", inplace=True) # this fills empty values with NaN
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y") # turn the date into an actual date-time date


C:\Users\Henry\AppData\Local\Temp\ipykernel_16644\3101903382.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NaN", inplace=True) # this fills empty values with NaN


In [118]:
geolocator = Nominatim(user_agent='postcode_checker')

# this function checks whether the postcode is a string & if it contains a number. If it contains a number it's likely a postcode so pass the str into the next function
def is_postcode(value):
    return isinstance(value, str) and any(character.isdigit() for character in value)


# The following function uses the postcode past into get_town & returns the 3rd from last value in the returned list. Which is often the town
def get_display_name(value):
    if is_postcode(value):
        location = geolocator.geocode(value, exactly_one=True)
        if location:
            display_name = location.raw.get("display_name", "")
            return display_name
        else:
            return value
    return value

# The following function parses the display_name from the comma and returns the -3 item in the list, if it is longer than 3 items long, as this is usually the town. If not then just returns the original value.
def get_town(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-3]  
    else:
        return value[0]


# The following function also parses the display_name from the comma and returns the -1 item which is usually the country. If not then returns NaN
def get_country(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-1]  
    else:
        return "NaN"



In [ ]:
df["displayName"] = df["locationName"].apply(get_display_name)


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,displayName
0,54590610,312930,The Clancy Group,NaN,NaN,Planner,NN175UA,NaN,NaN,NaN,16/04/2025,2025-03-05,"Planner 26,000 Corby We Care That’s why we nee...",9,https://www.reed.co.uk/jobs/planner/54590610,"NN17 5UA, North Northamptonshire, England, Uni..."
1,54537437,312930,The Clancy Group,NaN,NaN,Planner,EH545FD,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Livingston We Care That’s why we need ...,13,https://www.reed.co.uk/jobs/planner/54537437,"EH54 5FD, Houstoun Industrial Estate, Knightsr..."
2,54537415,312930,The Clancy Group,NaN,NaN,Planner,NN39BX,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Competitive Salary and Package Great B...,22,https://www.reed.co.uk/jobs/planner/54537415,"NN3 9BX, Crow Lane Industrial Estate, Northamp..."
3,54524565,677500,W B Power Services Limited,NaN,NaN,Project Planner,Heanor Gate,55000.0,70000.0,GBP,07/04/2025,2025-02-24,Job description Project Planner At WB Power Se...,42,https://www.reed.co.uk/jobs/project-planner/54...,Heanor Gate
4,54537279,437172,Matchtech,NaN,NaN,Planner OHL,Reading,45000.0,55000.0,GBP,09/04/2025,2025-02-26,"Our client, a leading organisation in the ener...",15,https://www.reed.co.uk/jobs/planner-ohl/54537279,Reading
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,54568777,477354,Michael Page Procurement & Supply Chain,NaN,NaN,Production Planner,Rochdale,35000.0,40000.0,GBP,17/03/2025,2025-03-03,A diligent Production Planner is required to j...,12,https://www.reed.co.uk/jobs/production-planner...,Rochdale
96,54577924,432293,MERJE Ltd,NaN,NaN,Financial Planner,ST136DS,NaN,NaN,NaN,15/04/2025,2025-03-04,"Our client, a&nbsp;well-established financial ...",1,https://www.reed.co.uk/jobs/financial-planner/...,"ST13 6DS, Leek, Staffordshire Moorlands, Staff..."
97,54524908,632887,Streamline Search Ltd,NaN,NaN,Transport Planner,Surrey,35000.0,41000.0,GBP,07/04/2025,2025-02-24,Streamline Search are working with a well esta...,4,https://www.reed.co.uk/jobs/transport-planner/...,Surrey
98,54503292,632887,Streamline Search Ltd,NaN,NaN,Transport Planner,Chertsey,35000.0,41000.0,GBP,02/04/2025,2025-02-19,Streamline Search are working with a well esta...,9,https://www.reed.co.uk/jobs/transport-planner/...,Chertsey


In [121]:
df["townName"] = df["displayName"].apply(get_town)
df["countryName"] = df["displayName"].apply(get_country)